In [1]:
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import datetime
import numpy as np

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import os
import datetime
import pandas as pd

class MyspiderbotSpider(scrapy.Spider):
    def readAndInsert(self,fileLocation):
        excelFile=pd.ExcelFile(fileLocation)
        
    name = 'mySpiderBot'
    allowed_domains = ['http://mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    errorFileName="error.log"
    dataFileName="Company_Registered_in_Last_30days_"+datetime.datetime.now().strftime("%d_%m_%h")+".xlsx"
    dataFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/spiders/fetchedFile/"+dataFileName
    def errorback(self,res):
        with open(self.errorFileName,"a") as fp:
            fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+str(res)+"\r\n")
        print("please check error log file for more information")
    def callback(self,res):
        content=res.body
        with open(self.dataFileLocation,"wb") as fp:
            fp.write(content)
        print("done writing")




    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url,headers=headers,errback=self.errorback,callback=self.callback,dont_filter=True)


In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import datetime
import numpy as np

class MyspiderbotSpider(scrapy.Spider):
        
    name = 'mySpiderBot'
    allowed_domains = ['http://mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    errorFileName="error.log"
    dataFileName="Company_Registered_in_Last_30days_"+datetime.datetime.now().strftime("%d_%m_%h")+".xlsx"
    dataFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders/fetchedFile/"+dataFileName
    successFileName="success.log"
    successFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders/"+successFileName
    errorFileName="error.log"
    errorFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders/"+errorFileName
    schemaColumns={
        "COMPANY NAME":"name",
        "LIMITED LIABILITY PARTNERSHIP NAME":"name",
        "FOREIGN LIMITED LIABILITY PARTNERSHIP":"name",
        "CIN":"identityNumber",
        "FCRN":"identityNumber",
        "LLPIN":"identityNumber",
        "FLLPIN":"identityNumber",
        "COUNTRY":"country",
        "COUNTRY OF INCORPORATION":"country",
        "DATE OF INCORPORATION":"dateOfIncorporation",
        "DATE OF REGISTRATION":"dateOfIncorporation",
        "STATE":"state",
        "STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA":"state",
        "ROC":"ROC",
        "CATEGORY":"category",
        "SUB CATEGORY":"subCategory",
        "CLASS":"class",
        "AUTHORIZED_CAPITAL":"authorizedCapital",
        "PAID CAPITAL":"paidCapital",
        "NUMBER OF PARTNERS":"numberOfPartners",
        "NUMBER OF MEMBERS (Applicable only in case of Co. without share Capital)":"numberOfPartners",
        "NUMBER OF DESIGNATED PARTNERS":"numberOfDesignatedParters",
        "ACTIVITY DESCRIPTION":"activityDescription",
        "ACTIVITY_DESCRIPTION":"activityDescription",
        "REGISTERED_OFFICE_ADDRESS":"officeAddressInIndia",
        "FOREIGN COMPANY PRESENT ADDRESS IN INDIA":"officeAddressInIndia",
        "FOREIGN OFFICE ADDRESS":"officeAddressInForeign",
        "TOTAL OBLIGATION OF CONTRIBUTION":"totalObligationOfContribution",
        "TYPE OF OFFICE":"typeOfOffice"}
    schema2sheet={
        "name":0,
        "identityNumber" :0,
        "country":0,
        "dateOfIncorporation":0,
        "state":0,
        "ROC":0,
        "category":0,
        "subCategory":0,
        "class":0,
        "authorizedCapital":0,
        "paidCapital":0,
        "numberOfPartners":0,
        "numberOfDesignatedParters":0,
        "activityDescription":0,
        "officeAddressInIndia":0,
        "officeAddressInForeign":0,
        "totalObligationOfContribution":0,
        "typeOfOffice":0
        }

    def createDataList(self,df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour,meta):
        if sheet == "Indian Companies Registered": #"Indian Companies Registered"
            columns=df.columns.values
            entity={}
            incorporationData={}
            emptyField={}
            registrationData={"registrationData":"null"}
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp['country']="India"
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=False;tmp1['isDomestic']=True;
                tmp1['identityLabel']="CIN"
                tmp1['type']="incorporation"
                row=df.iloc[index]
                for col in columns:
                    tmp['ROC']=row['ROC']
                    tmp['STATE']=row['STATE']
                    tmp1['vehicle']="company"
                    if col=="COMPANY NAME" or  col == "ACTIVITY_DESCRIPTION" or col == "CIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col]==not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "CATEGORY" or col == "SUB CATEGORY" or col == "CLASS" or col == "AUTHORIZED_CAPITAL" or col == "PAID CAPITAL" or col=="NUMBER OF MEMBERS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                entity['entity']=tmp1
                incorporationData['incorporationData']=tmp2
                tmp={**tmp,**entity,**incorporationData,**registrationData,**meta}
                dataList.append(tmp)
        
        elif sheet==sheetTwo: #Foreign Companies Registered
            columns=df.columns.values
            commonCol={}
            entity={}
            registrationData={}
            incorporationData={"incorporationData":"null"}
            emptyField={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                tmp['country']="India"
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                tmp1['identityLabel']="FCRN"
                tmp1['type']="registration"
                tmp1['vehicle']="company"
                tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FCRN"
                row=df.iloc[index]
                for col in columns:
                    tmp['ROC']=row['ROC']
                    tmp['STATE']=row['STATE']
                    if col=="COMPANY NAME" or col == "ACTIVITY_DESCRIPTION" or col == "FCRN" or col == "DATE OF REGISTRATION" or  col =="FOREIGN COMPANY PRESENT ADDRESS IN INDIA" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col]==not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "TYPE OF OFFICE" or col == "COUNTRY OF INCORPORATION" or col == "FOREIGN OFFICE ADDRESS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col]is not pd.np.nan else "null"
                
                    #tmp[schemaColumns[col]]=row[col] if row[col]==not pd.np.nan else "null"
                
                entity['entity']=tmp1
                registrationData['registrationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**tmp,**entity,**registrationData,**incorporationData,**emptyField,**meta}
                dataList.append(tmp)
            
        elif sheet==sheetThree: #LLP Registered
            columns=df.columns.values
            entity={}
            incorporationData={}
            emptyField={}
            registrationData={"registrationData":"null"}
            commonCol={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp['country']="India"
                tmp1['isCompany']=False;tmp1['isLLP']=True;tmp1['isForeign']=False;tmp1['isDomestic']=True;
                tmp1['identityLabel']="LLPIN"
                tmp1['type']="registration"
                row=df.iloc[index]
                for col in columns:
                    tmp['ROC']=row['ROC']
                    tmp['STATE']=row['STATE']
                    tmp1['vehicle'] = "LLP"
                    if col=="LIMITED LIABILITY PARTNERSHIP NAME" or  col == "ACTIVITY_DESCRIPTION" or col == "LLPIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col]==not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "TOTAL OBLIGATION OF CONTRIBUTION" or col=="NUMBER OF PARTNERS" or col=="NUMBER OF DESIGNATED PARTNERS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                entity['entity']=tmp1
                incorporationData['incorporationData']=tmp2
                tmp={**tmp,**entity,**incorporationData,**registrationData,**meta}
                dataList.append(tmp)

        
        elif sheet==sheetFour: #Foreign LLP Registered
            columns=df.columns.values
            entity={}
            incorporationData={"incorporationData":"null"}
            emptyField={}
            registrationData={}
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            try:
                for index in range(df.shape[0]):
                    tmp={}
                    tmp1={}
                    tmp2={}
                    tmp3={}
                    #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                    tmp['country']="India"
                    tmp1['isCompany']=False;tmp1['isLLP']=True;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                    tmp1['identityLabel']="FLLPIN"
                    tmp1['type']="registration"
                    row=df.iloc[index]
                    for col in columns:
                        tmp['ROC']=row['ROC']
                        tmp['STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA']=row['STATE'] if row['STATE'] is not pd.np.nan else "null"
                        tmp1['vehicle'] = "LLP"
                        if col=="FOREIGN LIMITED LIABILITY PARTNERSHIP" or  col == "ACTIVITY_DESCRIPTION" or col == "FLLPIN" or col == "DATE OF REGISTRATION" or col == "FOREIGN COMPANY PRESENT ADDRESS IN INDIA" :
                            #tmp[schemaColumns[col]]=row[col] if row[col]==not pd.np.nan else "null"
                            tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        elif col=="TYPE OF OFFICE" or col=="COUNTRY OF INCORPORATION" or col=="FOREIGN OFFICE ADDRESS":
                            tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                    entity['entity']=tmp1
                    registrationData['registrationData']=tmp2
                    tmp={**tmp,**entity,**incorporationData,**registrationData,**meta}
                    dataList.append(tmp)

            except Exception as e:
                pass
        return dataList
    
    def errorback(self,res):
        with open(self.errorFileName,"a") as fp:
            fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+str(res)+"\r\n")
        print("please check error log file for more information")
        
    def callback(self,res):
        content=res.body
        with open(self.dataFileLocation,"wb") as fp:
            fp.write(content)
        print("done writing")
        dataList=[]
        excelFile=pd.ExcelFile(self.dataFileLocation)
        sheetOne="Indian Companies Registered"
        sheetTwo="Foreign Companies Registered"
        sheetThree="LLP Registered"
        sheetFour="Foreign LLP Registered"
        meta={"meta":{}}
        meta['meta']["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta['meta']["source"]="http://mca.gov.in"
        meta['meta']["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        for sheet in excelFile.sheet_names:
            df=pd.read_excel(excelFile,sheet_name=sheet)
            df.columns=df.iloc[0]
            df.drop(0,inplace=True)
            dataList=self.createDataList(df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour,meta)
        mongoClient=MongoClient()
        db=mongoClient.mcaGovIn
        collection=db.companies
        
        try:
            collection.insert_many(dataList)
            with open(self.successFileLocation,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Successful, Successfully Crawled"+"\r\n")
            print("**********************************Successfully Crawled*******************************************")
        except Exception as e:
            with open(self.errorFileLocation,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Failed: - "+str(e)+"\r\n")
            print("**********************************Insertion Failed, Please check error log for more information*******************************************")
        




    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url,headers=headers,errback=self.errorback,callback=self.callback,dont_filter=True)

In [3]:
excelFile=pd.ExcelFile("./record.xlsx")
sheetOne="Indian Companies Registered"
sheetTwo="Foreign Companies Registered"
sheetThree="LLP Registered"
sheetFour="Foreign LLP Registered"

In [274]:
df=pd.read_excel(excelFile,sheet_name=sheetTwo)
df.columns=df.iloc[0]
df.drop(0,inplace=True)

In [18]:
excelFile=pd.ExcelFile("./record.xlsx")
sheetOne="Indian Companies Registered"
sheetTwo="Foreign Companies Registered"
sheetThree="LLP Registered"
sheetFour="Foreign LLP Registered"
schemaColumns={
        "COMPANY NAME":"name",
        "LIMITED LIABILITY PARTNERSHIP NAME":"name",
        "FOREIGN LIMITED LIABILITY PARTNERSHIP":"name",
        "CIN":"identityNumber",
        "FCRN":"identityNumber",
        "LLPIN":"identityNumber",
        "FLLPIN":"identityNumber",
        "COUNTRY":"country",
        "COUNTRY OF INCORPORATION":"country",
        "DATE OF INCORPORATION":"dateOfIncorporation",
        "DATE OF REGISTRATION":"creationDate",
        "STATE":"state",
        "STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA":"state",
        "ROC":"ROC",
        "CATEGORY":"category",
        "SUB CATEGORY":"subCategory",
        "CLASS":"class",
        "AUTHORIZED_CAPITAL":"authorizedCapital",
        "PAID CAPITAL":"paidCapital",
        "NUMBER OF PARTNERS":"numberOfPartners",
        "NUMBER OF MEMBERS (Applicable only in case of Co. without share Capital)":"numberOfPartners",
        "NUMBER OF DESIGNATED PARTNERS":"numberOfDesignatedParters",
        "ACTIVITY DESCRIPTION":"activityDiscription",
        "ACTIVITY_DESCRIPTION":"activityDiscription",
        "REGISTERED_OFFICE_ADDRESS":"officeAddressInIndia",
        "FOREIGN COMPANY PRESENT ADDRESS IN INDIA":"officeAddressInIndia",
        "FOREIGN OFFICE ADDRESS":"officeAddressInForeign",
        "TOTAL OBLIGATION OF CONTRIBUTION":"totalObligationOfContribution",
        "TYPE OF OFFICE":"typeOfOffice"}
    
schema2sheet={
    "name":0,
    "identityNumber" :0,
    "country":0,
    "dateOfIncorporation":0,
    "state":0,
    "ROC":0,
    "category":0,
    "subCategory":0,
    "class":0,
    "authorizedCapital":0,
    "paidCapital":0,
    "numberOfPartners":0,
    "numberOfDesignatedParters":0,
    "activityDiscription":0,
    "officeAddressInIndia":0,
    "officeAddressInForeign":0,
    "totalObligationOfContribution":0,
    "typeOfOffice":0
}
def createDataList(df,sheet,dataList):
    if sheet == "Indian Companies Registered": #"Indian Companies Registered"
        columns=df.columns.values
        entity={}
        incorporationData={}
        emptyField={}
        registrationData={"registrationData":"null"}
        commonCol={}
        for col in columns:
                commonCol[schemaColumns[col]]=0
        for index in range(df.shape[0]):
            tmp={}
            tmp1={}
            tmp2={}
            tmp3={}
            #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
            tmp['country']="India"
            tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=False;tmp1['isDomestic']=True;
            tmp1['identityLabel']="CIN"
            tmp1['type']="incorporation"
            row=df.iloc[index]
            for col in columns:
                tmp['ROC']=row['ROC']
                tmp['STATE']=row['STATE']
                tmp['vehicle']="company"
                if col=="COMPANY NAME" or  col == "ACTIVITY_DESCRIPTION" or col == "CIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    tmp1[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                elif col == "CATEGORY" or col == "SUB CATEGORY" or col == "CLASS" or col == "AUTHORIZED_CAPITAL" or col == "PAID CAPITAL" or col is "NUMBER OF MEMBERS":
                    tmp2[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
            entity['entity']=tmp1
            incorporationData['incorporationData']=tmp2
            tmp={**entity,**incorporationData,**registrationData}
            dataList.append(tmp)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
        
    elif sheet==sheetTwo: #Foreign Companies Registered
        columns=df.columns.values
        commonCol={}
        entity={}
        registrationData={}
        incorporationData={"incorporationData":"null"}
        emptyField={}
        for col in columns:
                commonCol[schemaColumns[col]]=0
        for index in range(df.shape[0]):
            tmp={}
            tmp1={}
            tmp2={}
            tmp3={}
            tmp['country']="India"
            tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=True;tmp1['isDomestic']=False;
            tmp1['identityLabel']="FCRN"
            tmp1['type']="registration"
            tmp1['vehicle']="company"
            tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FCRN"
            row=df.iloc[index]
            for col in columns:
                tmp['ROC']=row['ROC']
                tmp['STATE']=row['STATE']
                if col=="COMPANY NAME" or col == "ACTIVITY_DESCRIPTION" or col == "FCRN" or col == "DATE OF REGISTRATION" or  col =="FOREIGN COMPANY PRESENT ADDRESS IN INDIA" or col == "REGISTERED_OFFICE_ADDRESS" :
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    tmp1[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                elif col == "TYPE OF OFFICE" or col == "COUNTRY OF INCORPORATION" or col == "FOREIGN OFFICE ADDRESS":
                    tmp2[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                
            for unInsertedKey in schema2sheet.keys():
                if unInsertedKey not in commonCol:
                    #tmp[unInsertedKey]="null"
                    tmp3[unInsertedKey]="null"
            entity['entity']=tmp1
            registrationData['registrationData']=tmp2
            emptyField['emptyField']=tmp3
            tmp={**entity,**registrationData,**incorporationData,**emptyField}
            dataList.append(tmp)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
            
    elif sheet==sheetThree: #LLP Registered
        columns=df.columns.values
        entity={}
        incorporationData={}
        emptyField={}
        registrationData={"registrationData":"null"}
        commonCol={}
        for col in columns:
                commonCol[schemaColumns[col]]=0
        for index in range(df.shape[0]):
            tmp={}
            tmp1={}
            tmp2={}
            tmp3={}
            #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
            tmp['country']="India"
            tmp1['isCompany']=False;tmp1['isLLP']=True;tmp1['isForeign']=False;tmp1['isDomestic']=True;
            tmp1['identityLabel']="LLPIN"
            tmp1['type']="registration"
            row=df.iloc[index]
            for col in columns:
                tmp['ROC']=row['ROC']
                tmp['STATE']=row['STATE']
                tmp['vehicle'] = "LLP"
                if col=="LIMITED LIABILITY PARTNERSHIP NAME" or  col == "ACTIVITY_DESCRIPTION" or col == "LLPIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    tmp1[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                elif col is "TOTAL OBLIGATION OF CONTRIBUTION" or col is "NUMBER OF PARTNERS" or col is "NUMBER OF DESIGNATED PARTNERS":
                    tmp2[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
            entity['entity']=tmp1
            incorporationData['incorporationData']=tmp2
            tmp={**entity,**incorporationData,**registrationData}
            dataList.append(tmp)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
            
            
    elif sheet==sheetFour: #Foreign LLP Registered
        columns=df.columns.values
        entity={}
        incorporationData={"incorporationData":"null"}
        emptyField={}
        registrationData={}
        commonCol={}
        for col in columns:
                commonCol[schemaColumns[col]]=0
        try:
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp['country']="India"
                tmp1['isCompany']=False;tmp1['isLLP']=True;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                tmp1['identityLabel']="FLLPIN"
                tmp1['type']="registration"
                row=df.iloc[index]
                for col in columns:
                    tmp['ROC']=row['ROC']
                    tmp['STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA']=row['STATE'] if row['STATE'] is not pd.np.nan else "null"
                    tmp['vehicle'] = "LLP"
                    if col=="FOREIGN LIMITED LIABILITY PARTNERSHIP" or  col == "ACTIVITY_DESCRIPTION" or col == "FLLPIN" or col == "DATE OF REGISTRATION" or col == "FOREIGN COMPANY PRESENT ADDRESS IN INDIA" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col is "TYPE OF OFFICE" or col is "COUNTRY OF INCORPORATION" or col is "FOREIGN OFFICE ADDRESS":
                        tmp2[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                entity['entity']=tmp1
                registrationData['registrationData']=tmp2
                tmp={**entity,**incorporationData,**registrationData}
                dataList.append(tmp)
            meta={}
            meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
            meta["source"]="http://mca.gov.in"
            meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
            dataList.append({"metaData":meta})

        except Exception as e:
            pass
    return dataList
            
'''
df=pd.read_excel(excelFile,sheet_name="LLP Registered")
df.columns=df.iloc[0]
df.drop(0,inplace=True)
createDataList(df,sheet) 
'''
dataList=[]

for sheet in excelFile.sheet_names:
    df=pd.read_excel(excelFile,sheet_name=sheet)
    df.columns=df.iloc[0]
    df.drop(0,inplace=True)
    dataList=createDataList(df.head(1),sheet,dataList)
#df=pd.read_excel(excelFile,sheet_name="Indian Companies Registered")
#df.columns=df.iloc[0]
#df.drop(0,inplace=True)
#dataList=[]
#dataList=createDataList(df.head(),"Indian Companies Registered",dataList)


In [19]:
dataList

[{'entity': {'isCompany': True,
   'isLLP': False,
   'isForeign': False,
   'isDomestic': True,
   'identityLabel': 'CIN',
   'type': 'incorporation',
   'identityNumber': 'U01100AP2018PTC109756',
   'name': 'BUBBA AQUA FARM TECH PRIVATE LIMITED',
   'dateOfIncorporation': '2018-11-16',
   'activityDiscription': 'Activities Agriculture, Hunting and related Service activities',
   'officeAddressInIndia': '   10-2, OLD ANDHRA BANK ROAD, D2 CHIRALA CHIRALA Prakasam AP 523165 IN'},
  'incorporationData': {'category': 'Company limited by Shares',
   'subCategory': 'Non-govt company',
   'class': 'Private',
   'authorizedCapital': 100000,
   'paidCapital': 100000},
  'registrationData': 'null'},
 {'metaData': {'dateCreated': '11/12/18',
   'source': 'http://mca.gov.in',
   'sourceURL': 'http://mca.gov.in/mcafoportal/companiesRegReport.do'}},
 {'entity': {'isCompany': True,
   'isLLP': False,
   'isForeign': True,
   'isDomestic': False,
   'identityLabel': 'FCRN',
   'type': 'registration',

In [59]:
mongoClient=MongoClient()
db=mongoClient.mcaGovIn
collection=db.companies

In [60]:
collection.drop()
collection.insert_many(dataList)

In [ ]:
isCompany: True / False
isLLP: True / False
isForeign: True / False
isDomestic: True / False

In [112]:

"backup file*************************************************************************************"

# -*- coding: utf-8 -*-
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import datetime
import numpy as np

class MyspiderbotSpider(scrapy.Spider):
        
    name = 'mySpiderBot'
    allowed_domains = ['http://mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    errorFileName="error.log"
    dataFileName="Company_Registered_in_Last_30days_"+datetime.datetime.now().strftime("%d_%m_%h")+".xlsx"
    dataFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/spiders/fetchedFile/"+dataFileName
    successFileName="success.log"
    
    schemaColumns={
        "COMPANY NAME":"name",
        "LIMITED LIABILITY PARTNERSHIP NAME":"name",
        "FOREIGN LIMITED LIABILITY PARTNERSHIP":"name",
        "CIN":"identityNumber",
        "FCRN":"identityNumber",
        "LLPIN":"identityNumber",
        "FLLPIN":"identityNumber",
        "COUNTRY":"country",
        "COUNTRY OF INCORPORATION":"country",
        "DATE OF INCORPORATION":"dateOfIncorporation",
        "DATE OF REGISTRATION":"dateOfIncorporation",
        "STATE":"state",
        "STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA":"state",
        "ROC":"ROC",
        "CATEGORY":"category",
        "SUB CATEGORY":"subCategory",
        "CLASS":"class",
        "AUTHORIZED_CAPITAL":"authorizedCapital",
        "PAID CAPITAL":"paidCapital",
        "NUMBER OF PARTNERS":"numberOfPartners",
        "NUMBER OF MEMBERS (Applicable only in case of Co. without share Capital)":"numberOfPartners",
        "NUMBER OF DESIGNATED PARTNERS":"numberOfDesignatedParters",
        "ACTIVITY DESCRIPTION":"activityDiscription",
        "ACTIVITY_DESCRIPTION":"activityDiscription",
        "REGISTERED_OFFICE_ADDRESS":"officeAddressInIndia",
        "FOREIGN COMPANY PRESENT ADDRESS IN INDIA":"officeAddressInIndia",
        "FOREIGN OFFICE ADDRESS":"officeAddressInForeign",
        "TOTAL OBLIGATION OF CONTRIBUTION":"totalObligationOfContribution",
        "TYPE OF OFFICE":"typeOfOffice"}
    schema2sheet={
        "name":0,
        "identityNumber" :0,
        "country":0,
        "dateOfIncorporation":0,
        "state":0,
        "ROC":0,
        "category":0,
        "subCategory":0,
        "class":0,
        "authorizedCapital":0,
        "paidCapital":0,
        "numberOfPartners":0,
        "numberOfDesignatedParters":0,
        "activityDiscription":0,
        "officeAddressInIndia":0,
        "officeAddressInForeign":0,
        "totalObligationOfContribution":0,
        "typeOfOffice":0
        }

    def createDataList(self,df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour):
        if sheet==sheetOne: #"Indian Companies Registered"
            columns=df.columns.values
            entity={}
            incorporationData={}
            emptyField={}
            registrationData={"registrationData":"null"}
            commonCol={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=False;tmp1['isDomestic']=True;
                tmp1['identityLabel']="CIN"
                tmp1['type']="incorporation"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or  col == "ACTIVITY_DESCRIPTION" or col=="ROC" or col=="STATE" or col == "CIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "CATEGORY" or col == "SUB CATEGORY" or col == "CLASS" or col == "AUTHORIZED_CAPITAL" or col == "PAID CAPITAL" or col is "NUMBER OF MEMBERS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                incorporationData['incorporationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**incorporationData,**emptyField,**registrationData}
                dataList.append(tmp)
            
            
            
            
            
        
        elif sheet==sheetTwo: #Foreign Companies Registered
            columns=df.columns.values
            commonCol={}
            entity={}
            registrationData={}
            incorporationData={"incorporationData":"null"}
            emptyField={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                tmp1['identityLabel']="FCRN"
                tmp1['type']="registration"
                tmp1['type']="India"
                tmp1['vehicle']="company"
                tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FCRN"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or col == "ACTIVITY_DESCRIPTION" or col=="STATE" or col=="ROC" or col == "FCRN" or col == "DATE OF REGISTRATION" or  col =="FOREIGN COMPANY PRESENT ADDRESS IN INDIA" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "TYPE OF OFFICE" or col == "COUNTRY OF INCORPORATION" or col == "FOREIGN OFFICE ADDRESS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        if unInsertedKey=="COUNTRY":continue
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                registrationData['registrationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**registrationData,**incorporationData,**emptyField}
                dataList.append(tmp)
            
        elif sheet==sheetThree: #LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['vehicle']="LLP";tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="LLPIN"
                row=df.iloc[index]
                for col in columns:
                    tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        tmp[unInsertedKey]="null"
                dataList.append(tmp)        
        
        elif sheet==sheetFour: #Foreign LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['LLP']="company";tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FLLPIN"
                try:
                    row=df.iloc[index]
                    for col in columns:
                        tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    for unInsertedKey in self.schema2sheet.keys():
                        if unInsertedKey not in commonCol:
                            tmp[unInsertedKey]="null"
                    dataList.append(tmp)
                except Exception as e:
                    pass
        return dataList
    
    
    
    
    def readAndInsert(self,fileLocation):
        excelFile=pd.ExcelFile(fileLocation)

    def errorback(self,res):
        with open(self.errorFileName,"a") as fp:
            fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+str(res)+"\r\n")
        print("please check error log file for more information")
        
    def callback(self,res):
        content=res.body
        with open(self.dataFileLocation,"wb") as fp:
            fp.write(content)
        print("done writing")
        dataList=[]
        excelFile=pd.ExcelFile(self.dataFileLocation)
        sheetOne="Indian Companies Registered"
        sheetTwo="Foreign Companies Registered"
        sheetThree="LLP Registered"
        sheetFour="Foreign LLP Registered"
        for sheet in ["Indian Companies Registered","Foreign Companies Registered"]:
            df=pd.read_excel(excelFile,sheet_name=sheet)
            df.columns=df.iloc[0]
            df.drop(0,inplace=True)
            dataList=self.createDataList(df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
        mongoClient=MongoClient()
        db=mongoClient.mcaGovIn
        collection=db.companies
        
        try:
            collection.insert_many(dataList)
            with open(self.successFileName,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Successful, Successfully Crawled"+"\r\n")
            print("**********************************Successfully Crawled*******************************************")
        except Exception as e:
            with open(self.errorFileName,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Failed: - "+str(e)+"\r\n")
            print("**********************************Insertion Failed, Please check error log for more information*******************************************")
        




    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url,headers=headers,errback=self.errorback,callback=self.callback,dont_filter=True)

In [114]:
dataList

[{'isCompany': True,
  'isLLP': False,
  'isForeign': False,
  'isDomestic': True,
  'identityLabel': 'CIN',
  'identityNumber': 'U01100AP2018PTC109756',
  'name': 'BUBBA AQUA FARM TECH PRIVATE LIMITED',
  'dateOfIncorporation': '2018-11-16',
  'state': 'Andhra Pradesh',
  'ROC': 'RoC - Hyderabad',
  'category': 'Company limited by Shares',
  'subCategory': 'Non-govt company',
  'class': 'Private',
  'authorizedCapital': 100000,
  'paidCapital': 100000,
  'numberOfPartners': '0',
  'activityDiscription': 'Activities Agriculture, Hunting and related Service activities',
  'officeAddressInIndia': '   10-2, OLD ANDHRA BANK ROAD, D2 CHIRALA CHIRALA Prakasam AP 523165 IN',
  'country': 'null',
  'numberOfDesignatedParters': 'null',
  'officeAddressInForeign': 'null',
  'totalObligationOfContribution': 'null',
  'typeOfOffice': 'null'},
 {'isCompany': True,
  'isLLP': False,
  'isForeign': False,
  'isDomestic': True,
  'identityLabel': 'CIN',
  'identityNumber': 'U01100AP2018PTC109775',
  '

In [78]:
'COMPANY NAME' in schemaColumns

True

In [60]:
df.iloc[0]["DATE OF INCORPORATION"]

'2018-11-16'

In [53]:
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
#domains_='http://mca.gov.in/mcafoportal/companiesRegReport.do'
#start_urls='http://mca.gov.in/mcafoportal/companiesRegReport.do'
fileName="tmp.xlsx"
fileLocation="/home/digiapt/anaconda3/bin/webCrawler/mySpider/mySpider/spiders/"+fileName
class ScrapyspiderSpider(scrapy.Spider):
    #global domains_,start_urls
    name='scrapySpider'
    allowed_domains = ['http://www.mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    def errorFun(self,response):
        print("***************Please Check Your Internet Connection*******************")
    lastRow={}
    def callFun(self,response):
        scraped_info= response.body
        with open(fileLocation, 'wb') as f:
            f.write(scraped_info)
            f.flush()


        xls=pd.ExcelFile("./"+fileName)
        dataList=[]
        #{fileName:{last row of each datasheet}}
        lastRow={}
        jsonString=None
        with open("./lastRow.json",'r') as fp:
            jsonString=json.loads(fp.readlines()[0])
        for sheet in xls.sheet_names:
            df=pd.read_excel(xls,sheet_name=sheet)
            df.columns=df.iloc[0].values
            df.drop(0,inplace=True)
            if jsonString:
                lastRowJson=jsonString[sheet]
                if sheet=="LLP Registered":
                    try:
                        newDf=df[df.index[(df["LIMITED LIABILITY PARTNERSHIP NAME"]==lastRowJson["LIMITED LIABILITY PARTNERSHIP NAME"])&(df["LLPIN"]==lastRowJson["LLPIN"])].tolist()[0]:]
                        dataList.append({sheet:self.df2dict(newDf,sheet)})
                    except Exception as e:
                        pass
                elif sheet=="Foreign Companies Registered":
                    try:
                        newDf=df[df.index[(df["COMPANY NAME"]==lastRowJson["COMPANY NAME"])&(df["FCRN"]==lastRowJson["FCRN"])].tolist()[0]:]
                        dataList.append({sheet:self.df2dict(newDf,sheet)})
                    except Exception as e:
                        pass
                elif sheet=="Foreign LLP Registered":
                    try:
                        newDf=df[df.index[(df["FLLPIN"]==lastRowJson["FLLPIN"])&(df["FOREIGN LIMITED LIABILITY PARTNERSHIP"]==lastRowJson["FOREIGN LIMITED LIABILITY PARTNERSHIP"])].tolist()[0]:]
                        dataList.append({sheet:self.df2dict(newDf,sheet)})
                    except Exception as e:
                        pass
                elif sheet=="Indian Companies Registered":
                    try:
                        newDf=df[df.index[(df["COMPANY NAME"]==lastRowJson["COMPANY NAME"])&(df["CIN"]==lastRowJson["CIN"])].tolist()[0]:]
                        dataList.append({sheet:self.df2dict(newDf,sheet)})
                    except Exception as e:
                    pass                
            lastRow[sheet]=self.df2dict(df.tail(1),sheet)[0]

        '''
        df=pd.read_excel("./record.xlsx")
        df.columns=df.iloc[0].values
        df.drop(0,inplace=True)
        df.to_csv("record.csv",index=False)
        dfCsv=pd.read_csv("record.csv")
        fig= plt.figure(1, [20, 8])
        ax = fig.add_subplot(1, 1, 1) 
        ax.set_xlim(-1,100)
        plt.setp(ax.get_xticklabels(), fontsize=10, rotation='70')
        snsPlot=sns.countplot(df.STATE)
        snsPlot.figure.savefig("countPlot.jpg")
        '''
        

        client=MongoClient()
        db=client.Company_Registered_in_Last_30days
        collection=db.companies
        collection.insert_many(dataList)
        client.close()

        #with open("./lastRow.json",'w') as fp:
            #json.dump(lastRow,fp)
        #if os.path.exists("fileName"):
            #os.remove("filename")
        


        yield {"********************Crawling Finished********************":0}
    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url, headers=headers,errback=self.errorFun,callback=self.callFun,dont_filter=True)
    def df2dict(self,df,sheet):
        dataList=[]
        columns=df.columns
        for index in range(df.shape[0]):
            tmp={}
            for col in columns:
                if str(col).replace('.',' ').strip()=="DATE OF INCORPORATION":
                    parsed=str(df.iloc[index][col]).replace('.',' ').strip().split("-")
                    tmp[str(col).replace('.',' ').strip()]={"year":parsed[0],"month":parsed[1],"day":parsed[-1]}
                elif str(col).replace('.',' ').strip()=="REGISTERED_OFFICE_ADDRESS":
                    if sheet.strip()== "Indian Companies Registered":
                        parsed=str(df.iloc[index][col]).replace('.',' ').strip().rsplit(" ",2)
                        tmp[str(col).replace('.',' ').strip()]={"address":(parsed[0]+" "+parsed[-1]),"zipCode":parsed[-2]}
                    else:
                        parsed=str(df.iloc[index][col]).replace('.',' ').strip().rsplit(" ",1)
                        tmp[str(col).replace('.',' ').strip()]={"address":parsed[0],"zipCode":parsed[-1]}
                else:tmp[str(col).replace('.',' ').strip()]=str(df.iloc[index][col]).replace('.',' ')
            dataList.append(tmp)
        return dataList

IndentationError: expected an indented block (<ipython-input-53-2ade6452478d>, line 64)

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import datetime
import numpy as np

class MyspiderbotSpider(scrapy.Spider):
        
    name = 'mySpiderBot'
    allowed_domains = ['http://mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    errorFileName="error.log"
    dataFileName="Company_Registered_in_Last_30days_"+datetime.datetime.now().strftime("%d_%m_%h")+".xlsx"
    dataFileLocation="~/Desktop/mca_gov_in/mca_gov_in/spiders/fetchedFile/mca_gov_in/spiders/fetchedFile/"+dataFileName
    successFileName="success.log"
    
    schemaColumns={
        "COMPANY NAME":"name",
        "LIMITED LIABILITY PARTNERSHIP NAME":"name",
        "FOREIGN LIMITED LIABILITY PARTNERSHIP":"name",
        "CIN":"identityNumber",
        "FCRN":"identityNumber",
        "LLPIN":"identityNumber",
        "FLLPIN":"identityNumber",
        "COUNTRY":"country",
        "COUNTRY OF INCORPORATION":"country",
        "DATE OF INCORPORATION":"dateOfIncorporation",
        "DATE OF REGISTRATION":"dateOfIncorporation",
        "STATE":"state",
        "STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA":"state",
        "ROC":"ROC",
        "CATEGORY":"category",
        "SUB CATEGORY":"subCategory",
        "CLASS":"class",
        "AUTHORIZED_CAPITAL":"authorizedCapital",
        "PAID CAPITAL":"paidCapital",
        "NUMBER OF PARTNERS":"numberOfPartners",
        "NUMBER OF MEMBERS (Applicable only in case of Co. without share Capital)":"numberOfPartners",
        "NUMBER OF DESIGNATED PARTNERS":"numberOfDesignatedParters",
        "ACTIVITY DESCRIPTION":"activityDiscription",
        "ACTIVITY_DESCRIPTION":"activityDiscription",
        "REGISTERED_OFFICE_ADDRESS":"officeAddressInIndia",
        "FOREIGN COMPANY PRESENT ADDRESS IN INDIA":"officeAddressInIndia",
        "FOREIGN OFFICE ADDRESS":"officeAddressInForeign",
        "TOTAL OBLIGATION OF CONTRIBUTION":"totalObligationOfContribution",
        "TYPE OF OFFICE":"typeOfOffice"}
    schema2sheet={
        "name":0,
        "identityNumber" :0,
        "country":0,
        "dateOfIncorporation":0,
        "state":0,
        "ROC":0,
        "category":0,
        "subCategory":0,
        "class":0,
        "authorizedCapital":0,
        "paidCapital":0,
        "numberOfPartners":0,
        "numberOfDesignatedParters":0,
        "activityDiscription":0,
        "officeAddressInIndia":0,
        "officeAddressInForeign":0,
        "totalObligationOfContribution":0,
        "typeOfOffice":0
        }

    def createDataList(self,df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour):
        if sheet==sheetOne: #"Indian Companies Registered"
            columns=df.columns.values
            entity={}
            incorporationData={}
            emptyField={}
            registrationData={"registrationData":"null"}
            commonCol={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=False;tmp1['isDomestic']=True;
                tmp1['identityLabel']="CIN"
                tmp1['type']="incorporation"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or  col == "ACTIVITY_DESCRIPTION" or col=="ROC" or col=="STATE" or col == "CIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "CATEGORY" or col == "SUB CATEGORY" or col == "CLASS" or col == "AUTHORIZED_CAPITAL" or col == "PAID CAPITAL" or col is "NUMBER OF MEMBERS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                incorporationData['incorporationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**incorporationData,**emptyField,**registrationData}
                dataList.append(tmp)
            
            
            
            
            
        
        elif sheet==sheetTwo: #Foreign Companies Registered
            columns=df.columns.values
            commonCol={}
            entity={}
            registrationData={}
            incorporationData={"incorporationData":"null"}
            emptyField={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                tmp1['identityLabel']="FCRN"
                tmp1['type']="registration"
                tmp1['vehicle']="company"
                tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FCRN"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or col == "ACTIVITY_DESCRIPTION" or col=="STATE" or col=="ROC" or col == "FCRN" or col == "DATE OF REGISTRATION" or  col =="FOREIGN COMPANY PRESENT ADDRESS IN INDIA" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "TYPE OF OFFICE" or col == "COUNTRY OF INCORPORATION" or col == "FOREIGN OFFICE ADDRESS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                registrationData['registrationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**registrationData,**incorporationData,**emptyField}
                dataList.append(tmp)
            
        elif sheet==sheetThree: #LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['vehicle']="LLP";tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="LLPIN"
                row=df.iloc[index]
                for col in columns:
                    tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        tmp[unInsertedKey]="null"
                dataList.append(tmp)        
        
        elif sheet==sheetFour: #Foreign LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['LLP']="company";tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FLLPIN"
                try:
                    row=df.iloc[index]
                    for col in columns:
                        tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    for unInsertedKey in self.schema2sheet.keys():
                        if unInsertedKey not in commonCol:
                            tmp[unInsertedKey]="null"
                    dataList.append(tmp)
                except Exception as e:
                    pass
        return dataList
    
    

    def errorback(self,res):
        with open(self.errorFileName,"a") as fp:
            fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+str(res)+"\r\n")
        print("please check error log file for more information")

    def callback(self,res):
        content=res.body
        with open(self.dataFileLocation,"wb") as fp:
            fp.write(content)
        print("done writing")
        dataList=[]
        excelFile=pd.ExcelFile(self.dataFileLocation)
        sheetOne="Indian Companies Registered"
        sheetTwo="Foreign Companies Registered"
        sheetThree="LLP Registered"
        sheetFour="Foreign LLP Registered"
        for sheet in ["Indian Companies Registered","Foreign Companies Registered"]:
            df=pd.read_excel(excelFile,sheet_name=sheet)
            df.columns=df.iloc[0]
            df.drop(0,inplace=True)
            dataList=self.createDataList(df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
        mongoClient=MongoClient()
        db=mongoClient.mcaGovIn
        collection=db.companies
        
        try:
            collection.insert_many(dataList)
            with open(self.successFileName,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Successful, Successfully Crawled"+"\r\n")
            print("**********************************Successfully Crawled*******************************************")
        except Exception as e:
            with open(self.errorFileName,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Failed: - "+str(e)+"\r\n")
            print("**********************************Insertion Failed, Please check error log for more information*******************************************")
        

    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url,headers=headers,errback=self.errorback,callback=self.callback,dont_filter=True)

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import pandas as pd
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import datetime
import numpy as np

class MyspiderbotSpider(scrapy.Spider):
        
    name = 'mySpiderBot'
    allowed_domains = ['http://mca.gov.in']
    start_urls = ['http://mca.gov.in/mcafoportal/companiesRegReport.do']
    errorFileName="error.log"
    dataFileName="Company_Registered_in_Last_30days_"+datetime.datetime.now().strftime("%d_%m_%h")+".xlsx"
    dataFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders/fetchedFile/"+dataFileName
    successFileName="success.log"
    successFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders"+successFileName
    errorFileName="error.log"
    errorFileLocation="/home/digiapt/Desktop/mca_gov_in/mca_gov_in/mca_gov_in/spiders"+errorFileName
    
    schemaColumns={
        "COMPANY NAME":"name",
        "LIMITED LIABILITY PARTNERSHIP NAME":"name",
        "FOREIGN LIMITED LIABILITY PARTNERSHIP":"name",
        "CIN":"identityNumber",
        "FCRN":"identityNumber",
        "LLPIN":"identityNumber",
        "FLLPIN":"identityNumber",
        "COUNTRY":"country",
        "COUNTRY OF INCORPORATION":"country",
        "DATE OF INCORPORATION":"dateOfIncorporation",
        "DATE OF REGISTRATION":"dateOfIncorporation",
        "STATE":"state",
        "STATE OF PRINCIPAL PLACE OF BUSINESS IN INDIA":"state",
        "ROC":"ROC",
        "CATEGORY":"category",
        "SUB CATEGORY":"subCategory",
        "CLASS":"class",
        "AUTHORIZED_CAPITAL":"authorizedCapital",
        "PAID CAPITAL":"paidCapital",
        "NUMBER OF PARTNERS":"numberOfPartners",
        "NUMBER OF MEMBERS (Applicable only in case of Co. without share Capital)":"numberOfPartners",
        "NUMBER OF DESIGNATED PARTNERS":"numberOfDesignatedParters",
        "ACTIVITY DESCRIPTION":"activityDiscription",
        "ACTIVITY_DESCRIPTION":"activityDiscription",
        "REGISTERED_OFFICE_ADDRESS":"officeAddressInIndia",
        "FOREIGN COMPANY PRESENT ADDRESS IN INDIA":"officeAddressInIndia",
        "FOREIGN OFFICE ADDRESS":"officeAddressInForeign",
        "TOTAL OBLIGATION OF CONTRIBUTION":"totalObligationOfContribution",
        "TYPE OF OFFICE":"typeOfOffice"}
    schema2sheet={
        "name":0,
        "identityNumber" :0,
        "country":0,
        "dateOfIncorporation":0,
        "state":0,
        "ROC":0,
        "category":0,
        "subCategory":0,
        "class":0,
        "authorizedCapital":0,
        "paidCapital":0,
        "numberOfPartners":0,
        "numberOfDesignatedParters":0,
        "activityDiscription":0,
        "officeAddressInIndia":0,
        "officeAddressInForeign":0,
        "totalObligationOfContribution":0,
        "typeOfOffice":0
        }

    def createDataList(self,df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour):
        if sheet==sheetOne: #"Indian Companies Registered"
            columns=df.columns.values
            entity={}
            incorporationData={}
            emptyField={}
            registrationData={"registrationData":"null"}
            commonCol={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                #tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="CIN"
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=False;tmp1['isDomestic']=True;
                tmp1['identityLabel']="CIN"
                tmp1['type']="incorporation"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or  col == "ACTIVITY_DESCRIPTION" or col=="ROC" or col=="STATE" or col == "CIN" or col == "DATE OF INCORPORATION" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "CATEGORY" or col == "SUB CATEGORY" or col == "CLASS" or col == "AUTHORIZED_CAPITAL" or col == "PAID CAPITAL" or col is "NUMBER OF MEMBERS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
            
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                incorporationData['incorporationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**incorporationData,**emptyField,**registrationData}
                dataList.append(tmp)
            
            
            
            
            
        
        elif sheet==sheetTwo: #Foreign Companies Registered
            columns=df.columns.values
            commonCol={}
            entity={}
            registrationData={}
            incorporationData={"incorporationData":"null"}
            emptyField={}
            for col in columns:
                commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp1={}
                tmp2={}
                tmp3={}
                tmp1['isCompany']=True;tmp1['isLLP']=False;tmp1['isForeign']=True;tmp1['isDomestic']=False;
                tmp1['identityLabel']="FCRN"
                tmp1['type']="registration"
                tmp1['vehicle']="company"
                tmp['isCompany']=True;tmp['isLLP']=False;tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FCRN"
                row=df.iloc[index]
                for col in columns:
                    if col=="COMPANY NAME" or col == "ACTIVITY_DESCRIPTION" or col=="STATE" or col=="ROC" or col == "FCRN" or col == "DATE OF REGISTRATION" or  col =="FOREIGN COMPANY PRESENT ADDRESS IN INDIA" or col == "REGISTERED_OFFICE_ADDRESS" :
                        #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                        tmp1[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    elif col == "TYPE OF OFFICE" or col == "COUNTRY OF INCORPORATION" or col == "FOREIGN OFFICE ADDRESS":
                        tmp2[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                    #tmp[schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                
                
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        #tmp[unInsertedKey]="null"
                        tmp3[unInsertedKey]="null"
                entity['entity']=tmp1
                registrationData['registrationData']=tmp2
                emptyField['emptyField']=tmp3
                tmp={**entity,**registrationData,**incorporationData,**emptyField}
                dataList.append(tmp)
            
        elif sheet==sheetThree: #LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['vehicle']="LLP";tmp['isForeign']=False;tmp['isDomestic']=True;tmp['identityLabel']="LLPIN"
                row=df.iloc[index]
                for col in columns:
                    tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                for unInsertedKey in self.schema2sheet.keys():
                    if unInsertedKey not in commonCol:
                        tmp[unInsertedKey]="null"
                dataList.append(tmp)        
        
        elif sheet==sheetFour: #Foreign LLP Registered
            columns=df.columns.values
            commonCol={}
            for col in columns:
                    commonCol[self.schemaColumns[col]]=0
            for index in range(df.shape[0]):
                tmp={}
                tmp['isCompany']=False;tmp['isLLP']=True;tmp['LLP']="company";tmp['isForeign']=True;tmp['isDomestic']=False;tmp['identityLabel']="FLLPIN"
                try:
                    row=df.iloc[index]
                    for col in columns:
                        tmp[self.schemaColumns[col]]=row[col] if row[col] is not pd.np.nan else "null"
                    for unInsertedKey in self.schema2sheet.keys():
                        if unInsertedKey not in commonCol:
                            tmp[unInsertedKey]="null"
                    dataList.append(tmp)
                except Exception as e:
                    pass
        return dataList
    
    

    def errorback(self,res):
        with open(self.errorFileName,"a") as fp:
            fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+str(res)+"\r\n")
        print("please check error log file for more information")

    def callback(self,res):
        content=res.body
        with open(self.dataFileLocation,"wb") as fp:
            fp.write(content)
        print("done writing")
        dataList=[]
        excelFile=pd.ExcelFile(self.dataFileLocation)
        sheetOne="Indian Companies Registered"
        sheetTwo="Foreign Companies Registered"
        sheetThree="LLP Registered"
        sheetFour="Foreign LLP Registered"
        for sheet in ["Indian Companies Registered","Foreign Companies Registered"]:
            df=pd.read_excel(excelFile,sheet_name=sheet)
            df.columns=df.iloc[0]
            df.drop(0,inplace=True)
            dataList=self.createDataList(df,sheet,dataList,sheetOne,sheetTwo,sheetThree,sheetFour)
        meta={}
        meta["dateCreated"]=datetime.datetime.now().strftime("%d/%m/%y")
        meta["source"]="http://mca.gov.in"
        meta["sourceURL"]="http://mca.gov.in/mcafoportal/companiesRegReport.do"
        dataList.append({"metaData":meta})
        mongoClient=MongoClient()
        db=mongoClient.mcaGovIn
        collection=db.companies
        
        try:
            collection.insert_many(dataList)
            with open(self.successFileLocation,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Successful, Successfully Crawled"+"\r\n")
            print("**********************************Successfully Crawled*******************************************")
        except Exception as e:
            with open(self.errorFileLocation,"a") as fp:
                fp.write(datetime.datetime.now().strftime("%d-%m-%h (%H:%M:%S)")+" - "+" Insertion Failed: - "+str(e)+"\r\n")
            print("**********************************Insertion Failed, Please check error log for more information*******************************************")
        

    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0','Cache-Control': 'max-age=0','referer':'mca.gov.in'}
        for url in self.start_urls:
            yield scrapy.Request(url,headers=headers,errback=self.errorback,callback=self.callback,dont_filter=True)